# SIGNATURE VERIFICATION

In [1]:
########### THINNING ALGORITHM #############

from scipy import weave
import numpy as np
import cv2

def _thinningIteration(im, iter):
    I, M = im, np.zeros(im.shape, np.uint8)
    expr = """
    for (int i = 1; i < NI[0]-1; i++) {
        for (int j = 1; j < NI[1]-1; j++) {
            int p2 = I2(i-1, j);
            int p3 = I2(i-1, j+1);
            int p4 = I2(i, j+1);
            int p5 = I2(i+1, j+1);
            int p6 = I2(i+1, j);
            int p7 = I2(i+1, j-1);
            int p8 = I2(i, j-1);
            int p9 = I2(i-1, j-1);

            int A  = (p2 == 0 && p3 == 1) + (p3 == 0 && p4 == 1) +
                     (p4 == 0 && p5 == 1) + (p5 == 0 && p6 == 1) +
                     (p6 == 0 && p7 == 1) + (p7 == 0 && p8 == 1) +
            (p8 == 0 && p9 == 1) + (p9 == 0 && p2 == 1);
            int B  = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9;
            int m1 = iter == 0 ? (p2 * p4 * p6) : (p2 * p4 * p8);
            int m2 = iter == 0 ? (p4 * p6 * p8) : (p2 * p6 * p8);

            if (A == 1 && B >= 2 && B <= 6 && m1 == 0 && m2 == 0) {
                M2(i,j) = 1;
            }
        }
    } 
    """

    weave.inline(expr, ["I", "iter", "M"])
    return (I & ~M)


def thinning(src):
    dst = src.copy() / 255
    prev = np.zeros(src.shape[:2], np.uint8)
    diff = None

    while True:
        dst = _thinningIteration(dst, 0)
        dst = _thinningIteration(dst, 1)
        diff = np.absolute(dst - prev)
        prev = dst.copy()
        if np.sum(diff) == 0:
            break

    return dst * 255

############# ESCALAMIENTO #############
# Metodo que permite escalar la imagen a un tamaño por default(100)
# para que todas las imagenes posean el mismo tamaño
def scale(image, height=500):
    # Obtenemos las filas y columnas
    original_h, original_w = np.float32(image.shape[:2])
    # Calculamos el ancho que tendra con respecto al alto(100)
    width = int( height * ( original_w / original_h ) )
    # Escalamos la imagen
    image_scaled = cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA)
    # Retornamos la imagen escalada
    return image_scaled

def normalize(image):
    # dilating image
    img = image.copy()
    kernel = np.ones((80,80), np.uint8)
    img = cv2.erode(img, kernel, iterations = 1)

    # Binarization
    ret,thresh = cv2.threshold(img,127,255,0)
    _,contours,hierarchy = cv2.findContours(thresh, 1, 2)

    cnt = contours[0]

    # Bounding Rect
    x,y,w,h = cv2.boundingRect(cnt)

    # Cropping
    image = image[y:y+h,x:x+w]
    return image

############# PREPROCESSING #############
def preprocess(image, showProcess=False):
    #GrayScale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if showProcess:
        cv2.imshow(' GrayScale', image)
        cv2.waitKey(0)
    #Scaling
    image = scale(image)
    if showProcess:
        cv2.imshow(' Scaled', image)
        cv2.waitKey(0)
    #Denoising
    image = cv2.fastNlMeansDenoising(image, None, 6, 7, 21)
    if showProcess:
        cv2.imshow(' Denoised', image)
        cv2.waitKey(0)
    # Binarizacion
    ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
    if showProcess:
        cv2.imshow(' Binarizacion inv 1', image)
        cv2.waitKey(0)
    # Thinning
    image = thinning(image)
    if showProcess:
        cv2.imshow(' Thining', image)
        cv2.waitKey(0)
    # Binarizacion inv
    ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
    if showProcess:
        cv2.imshow(' Binarizacion inv 2', image)
        cv2.waitKey(0)
    # Normalizing
    image = normalize(image);
    if showProcess:
        cv2.imshow(' Normalized', image)
        cv2.waitKey(0)
    
    return image

def dilatex5(image):
    # DILATION 
    image_color_black = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image_color_red = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image_color_green= cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image_color_blue = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    image_color_black[image == 0] = [0, 0, 0]
    image_color_red[image == 0]   = [0, 0, 255]
    image_color_green[image == 0] = [0, 255, 0]
    image_color_blue[image == 0]  = [255, 0, 0]

    kernel_3 = np.ones((3,3), np.uint8)
    kernel_6 = np.ones((6,6), np.uint8)
    kernel_10 = np.ones((10,10), np.uint8) 
    kernel_16 = np.ones((16,16), np.uint8)

    image_3 = cv2.erode(image_color_black, kernel_3, iterations = 1)
    image_6 = cv2.erode(image_color_red, kernel_6, iterations = 1)
    image_10 = cv2.erode(image_color_green, kernel_10, iterations = 1)
    image_16 = cv2.erode(image_color_blue, kernel_16, iterations = 1)

    #cv2.imshow("Kernel 3", image_3)
    #cv2.imshow("Kernel 6", image_6)
    #cv2.imshow("Kernel 10", image_10)
    #cv2.imshow("Kernel 16", image_16)

    # FUSION
    img_fusion_1 = cv2.bitwise_and(image_16,image_10)
    img_fusion_1[cv2.threshold(cv2.cvtColor(img_fusion_1, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY)[1] == 0] = [0, 255, 0]

    img_fusion_2 = cv2.bitwise_and(img_fusion_1, image_6)
    img_fusion_2[cv2.threshold(cv2.cvtColor(img_fusion_2, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY)[1] == 0] = [0, 0, 255]

    img_fusion_3 = cv2.bitwise_and(img_fusion_2, image_3)

    return img_fusion_3


In [2]:
##### TESTING GENERAL ########

import cv2
import numpy as np
image = cv2.imread('images/signature.jpg')
image = preprocess(image)
cv2.imshow('Signature PreProcessed', image)
cv2.waitKey(0)

img_dilated = dilatex5(image)
cv2.imshow('Signature Dilated x5', img_dilated)

cv2.waitKey(0)

# FEATURES
imHeight, imWidth = np.float32(image.shape[:2])

# Aspect Ratio
aspectRatio = imHeight / imWidth
print 'Aspect Ratio : ', aspectRatio

# Density Ratio


# Ocupancy Ratio
## Negro 0 (cero)
## Otros(Blanco) NonZero
signaturePixels = cv2.countNonZero(image)
totalPixels = imHeight*imWidth
occupancyRatio = signaturePixels / totalPixels
print 'Occupancy Ratio : ', occupancyRatio

# Harris Corners
corners = cv2.cornerHarris(image, 3, 3, 0.05)
print 'Harris Corners : ' , len(corners)

cv2.destroyAllWindows()


Aspect Ratio :  0.652174
Occupancy Ratio :  0.989616747182
Harris Corners :  450


In [64]:
##### TESTING PREPROCESSING ########
import cv2
import numpy as np

image = cv2.imread('images/signature.jpg')
cv2.imshow('1.Signature Original', image)
cv2.waitKey(0)
# A ESCALA DE GRISES
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('2.Signature GrayScale', image)
cv2.waitKey(0)

# ESCALAMIENTO
image = scale(image)
cv2.imshow('3.Signature Scaled', image)
cv2.waitKey(0)

# ELIMINACION DE RUIDO
image = cv2.fastNlMeansDenoising(image, None, 6, 7, 21)
cv2.imshow('4.Signature Denoised', image)
cv2.waitKey(0)

# BINARIZACION <127 -> 0 (black), >=127 -> 255 (white)
ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('5. Threshold Binary', image)
cv2.waitKey(0)

# THINING
image = thinning(image)
cv2.imshow('5. Thinning', image)
cv2.waitKey(0)


# *** ROTACION ***

# *** EROSION ***
#kernel = np.ones((5,5), np.uint8)
#image = cv2.dilate(image, kernel, iterations = 1)
#cv2.imshow("7. Erosion", image)
#cv2.waitKey(0)

# *** DILATACION ***
#kernel = np.ones((5,5), np.uint8)
#image = cv2.erode(image, kernel, iterations = 1)
#cv2.imshow("6. Dilation", image)
#cv2.waitKey(0)

cv2.destroyAllWindows()

In [24]:
# Contours
import cv2
import numpy as np

img = cv2.imread('images/signature.jpg',0)

# dilating image
kernel = np.ones((50,50), np.uint8)
img = cv2.erode(img, kernel, iterations = 1)

ret,thresh = cv2.threshold(img,127,255,0)
_,contours,hierarchy = cv2.findContours(thresh, 1, 2)

# Moments
cnt = contours[0]
M = cv2.moments(cnt)
print M

#Centroid
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])
print 'contour area :', M['m00'] , cv2.contourArea(cnt)
print 'Contour perimeter : ', cv2.arcLength(cnt,True)
print 'centroid : ',cx,cy

# Contour Approximation
epsilon = 0.1*cv2.arcLength(cnt,True)
approx = cv2.approxPolyDP(cnt,epsilon,True)
# Hull
hull = cv2.convexHull(cnt)
# Bounding Rect
x,y,w,h = cv2.boundingRect(cnt)

#cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
img = img[y:y+h,x:x+w]

cv2.imshow('d',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

{'mu02': 1872255521.951889, 'mu03': -74853547521.00781, 'm11': 20634905060.958332, 'nu02': 0.07635346518329157, 'm12': 6859730461020.366, 'mu21': 98807478878.62793, 'mu20': 4381222472.423008, 'nu20': 0.17867300354369092, 'm30': 19723103372623.35, 'nu21': 0.01018285026168108, 'mu11': -197050284.63219452, 'mu12': -111927729580.68933, 'nu11': -0.00803601424624805, 'nu12': -0.011534990300178766, 'm02': 16019003624.083332, 'm03': 5865446726444.101, 'm00': 156591.5, 'm01': 47066554.0, 'mu30': 328092989471.91797, 'nu30': 0.03381243830544177, 'nu03': -0.007714218342706043, 'm10': 69308391.16666666, 'm20': 35057556028.916664, 'm21': 10461591394541.033}
contour area : 156591.5 156591.5
Contour perimeter :  3740.67739677
centroid :  442 300


In [7]:
cv2.imshow('half 1', image[:,:image.shape[1]/2])
cv2.imshow('half 2', image[:,image.shape[1]/2:])
cv2.waitKey(0)
cv2.destroyAllWindows()